In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import pandas as pd
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
!nvidia-smi

Thu Nov 12 10:38:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    34W /  70W |   2569MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 訓練期間的batch大小
batch_size = 64

# 測試期間的batch大小
test_batch_size = 32

# 訓練圖像的空間大小。所有圖像將使用resize調整為此大小。
image_size = 400

# label種類數
class_num = 196

# 訓練epochs的大小
num_epochs = 10

# 初始學習速率
lr = 0.01

# 你的資料夾路徑
dir_path = './drive/My Drive/HW1/'

# Data pre-processing

In [ ]:
# 讀取training data的id和label
df = pd.read_csv(dir_path + 'data/training_labels.csv')
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11185 entries, 0 to 11184
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      11185 non-null  int64 
 1   label   11185 non-null  object
dtypes: int64(1), object(1)
memory usage: 174.9+ KB
None
     id                                label
0  9350          Ford F-150 Regular Cab 2007
1  2645                      BMW X6 SUV 2012
2  2267              BMW 1 Series Coupe 2012
3  8553              Fisker Karma Sedan 2012
4  6990  Dodge Ram Pickup 3500 Crew Cab 2010


In [ ]:
# label encoder
label = df['label']
label_set = set(label)
print(len(label_set))
label_list = list(label_set)
label_dict = {}
for i in range(len(label_list)):
    label_dict[label_list[i]] = i

196


In [ ]:
# 處理圖片路徑
id = [i for i in df['id']]
for i in range(len(id)):
    if id[i] >= 10000:
        id[i] = '0' + str(id[i])
    elif id[i] >= 1000:
        id[i] = '00' + str(id[i])
    elif id[i] >= 100:
        id[i] = '000' + str(id[i])
    elif id[i] >= 10:
        id[i] = '0000' + str(id[i])
    else:
        id[i] = '00000' + str(id[i])

img_datas = [os.path.join(dir_path+"data/training_data", i+'.jpg') for i in id]
print(len(img_datas))

11185


In [ ]:
# 將label encode
label_datas = [label_dict[i] for i in df['label']]

# Load Data

In [ ]:
# 自定義資料集
class TrainDataset(Dataset):
    def __init__(self, imgs, labels, transform=None, target_transform=None):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        img = Image.open(self.imgs[index]).convert('RGB')
        label = self.labels[index]

        if self.transform:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.labels)

In [ ]:
train_data = TrainDataset(imgs=img_datas, labels=label_datas,
                          transform=transforms.Compose([
                            transforms.Resize((image_size, image_size)),
                            transforms.RandomRotation(20),
                            transforms.RandomHorizontalFlip(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5),
                                                 (0.5, 0.5, 0.5))
                            ]))

# 使用dataloader幫助分batch和shuffle
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

print(len(train_loader))

175


# ResNet

In [ ]:
# load pre-trained model
model = models.resnet34(pretrained=True)

# 修改最後一層輸出為196類
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, class_num)

# 丟到gpu
model = model.to(device)

# Loss & Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 使用ReduceLROnPlateau幫助自動根據正確率更新learning rate
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                 mode='max',
                                                 patience=3,
                                                 threshold=0.9,
                                                 verbose=True)

# Training

In [ ]:
def train(model, criterion, optimizer, scheduler, num_epochs):
    train_loss = 0.0
    train_correct = 0

    since = time.time()
    for epoch in range(num_epochs):
        model.train()

        print("Epoch:{}/{}".format(epoch+1, num_epochs))
        print("-"*10)

        for i, (imgs, labels) in enumerate(train_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            # parameter gradients歸零
            optimizer.zero_grad()

            # Forward and backward model
            outputs = model.forward(imgs)
            loss = criterion(outputs, labels)
            loss.backward()

            # 更新model weights
            optimizer.step()

            # 累計loss和預測正確數
            train_loss += loss.item()
            _, pred = torch.max(outputs, 1)
            train_correct += torch.sum(pred == labels)

        # 計算總共的loss和正確率
        train_loss = train_loss / len(train_loader)
        train_correct_cpu = train_correct.cpu()
        train_correct_np = train_correct_cpu.numpy()
        train_accuracy = 100*train_correct_np / len(img_datas)

        print("Train Loss: {} Train Accuracy: {}".format(train_loss,
                                                         train_accuracy))
        train_loss = 0.0
        train_correct = 0

        # 儲存model參數
        checkpoint = {'state_dict': model.state_dict(),
                      'opt_state': optimizer.state_dict}
        torch.save(checkpoint, dir_path+'temp_checkpoint.pth')

        # 根據正確率更新learning rate
        scheduler.step(train_accuracy)

        # 計算1個epoch花費時間
        now_time = time.time() - since
        print("Training time is:{:.0f}m {:.0f}s\n".format(now_time//60,
                                                          now_time % 60))
        since = time.time()

In [ ]:
train(model, criterion, optimizer, scheduler, num_epochs)

Epoch:1/10
----------
Train Loss: 4.117915620803833 Train Accuracy: 15.136343316942334
Training time is:6m 2s

Epoch:2/10
----------
Train Loss: 1.6999817834581648 Train Accuracy: 60.84041126508717
Training time is:6m 1s

Epoch:3/10
----------
Train Loss: 0.8018406122071402 Train Accuracy: 81.30531962449709
Training time is:6m 1s

Epoch:4/10
----------


# Save Model

In [ ]:
checkpoint = {'state_dict': model.state_dict(),
              'model': model,
              'opt_state': optimizer.state_dict}

torch.save(checkpoint, dir_path+'checkpoint.pth')

# Load Model

In [ ]:
model2 = models.resnet34(pretrained=True)
num_ftrs = model2.fc.in_features
model2.fc = nn.Linear(num_ftrs, class_num)
model2 = model2.to(device)

In [ ]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model2.load_state_dict(checkpoint['state_dict'], strict=False)

    return model2
model_test = load_checkpoint(dir_path+'checkpoint.pth')

# Load Testing Data

In [ ]:
# 取得該資料夾中所有檔案名
test_id = os.listdir(dir_path+'data/testing_data/')

# 處理圖片路徑
testing_imgs = [os.path.join(dir_path+"data/testing_data", i) for i in test_id]
print(testing_imgs)
print(len(testing_imgs))

In [ ]:
class TestDataset(Dataset):
    def __init__(self, imgs, id, transform=None, target_transform=None):
        self.imgs = imgs
        self.id = id
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        img = Image.open(self.imgs[index]).convert('RGB')
        id = self.id[index]
        if self.transform:
            img = self.transform(img)
        return img, id

    def __len__(self):
        return len(self.imgs)

In [ ]:
test_data = TestDataset(imgs=testing_imgs, id=test_id,
                        transform=transforms.Compose([
                          transforms.Resize((image_size, image_size)),
                          transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5),
                                               (0.5, 0.5, 0.5))
                          ]))

test_loader = DataLoader(test_data, batch_size=test_batch_size)
print(len(test_loader))

In [ ]:
def test(model):
    since = time.time()
    model.eval()
    pred_id = []
    pred_label = []

    with torch.no_grad():
        for i, (imgs, id) in enumerate(test_loader):
            imgs = imgs.to(device)

            # 取得預測機率
            outputs = model(imgs)

            # 轉為預測結果
            _, pred = torch.max(outputs, 1)
            pred_cpu = pred.cpu()
            pred_np = pred_cpu.numpy()

            for j in id:
                pred_id.append(int(j.split('.')[0]))
            for k in pred_np:
                pred_label.append(label_list[k])

            now_time = time.time() - since
            print("{}/{} test time is:{:.0f}m {:.0f}s".format(int(i+1),
                                                              len(test_loader),
                                                              now_time//60,
                                                              now_time % 60))
            since = time.time()

    print(len(pred_id))
    print(len(pred_label))

    return pred_id, pred_label

In [ ]:
pred_id, pred_label = test(model_test)

# to .csv

In [ ]:
dict1 = {"id": pred_id, "label": pred_label}

# 轉為dataframe再透過pandas轉成csv檔
result_df = pd.DataFrame(dict1)
result_df.to_csv(dir_path+'Result.csv', index=False)
print(result_df)

         id                               label
0     13311    Mercedes-Benz C-Class Sedan 2012
1     12654     Land Rover Range Rover SUV 2012
2     12930  Maybach Landaulet Convertible 2012
3     13120          McLaren MP4-12C Coupe 2012
4     12809         Lincoln Town Car Sedan 2011
...     ...                                 ...
4995   3387   Bentley Continental GT Coupe 2007
4996   3326         Bentley Mulsanne Sedan 2011
4997   3351   Bentley Continental GT Coupe 2012
4998   3043             BMW Z4 Convertible 2012
4999   3193           Bentley Arnage Sedan 2009

[5000 rows x 2 columns]
